In [1]:
import os
import glob
import ijson
import pandas as pd
import numpy as np

### Download the data from the eia.gov api 
##### Goal: capture the increase in renewable energy production relative to the total production, consumption, and population to see if there has been a substantial increase
##### Variables: All energy values are in billions of British Thermal Units (BTUs)
    Total Energy Production
    Total Renewable Production Other Than Fuel Ethanol
    Total Consumption
    Total Population (in thousands of people)

In [113]:
# Define what states and data we want to capture
states = ['AL','AK','AZ','CA','CO','CT','DE','DC','FL','GA','HI','ID','IL','IN','IA','KS','KY', \
          'LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND', \
          'OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

series = {'SEDS.TEPRB':['total_production','Total Energy Production'], \
          'SEDS.ROPRB':['total_renewable_production','Total Renewable Production Other Than Fuel Ethanal'], \
          'SEDS.TETCB':['total_consumption','Total Energy Consumption'], \
          'SEDS.TPOPP':['total_population','Total Resident Population']
          }

def api_call(state='AL', subfolder='total_production/', series_id='SEDS.TEPRB', API='e42cb93209b9e060fc525d850beceb55'):
    request = 'wget -O '+subfolder+'/'+state+'.json '+'"http://api.eia.gov/series/?api_key='+API+'&series_id='+series_id+'.'+state+'.A&out=json"' 
    return request

In [114]:
# Get data from API
for se in series.keys():
    for st in states:
        os.system(api_call(state=st, subfolder=series[se][0], series_id=se))

In [119]:
# Create a dataframe for each variable
df = pd.DataFrame()

for k in series.keys():
    filename = series[k][0]+'/'+s+'.json'
    with open(filename, 'r') as f:
        objects = list(ijson.items(f, 'series'))[0][0]['data']
        years = [i[0]+'_'+series[k][0] for i in objects] 
        years.insert(0,'State')
    
    values = list()
    for s in states:
        filename = series[k][0]+'/'+s+'.json'
        with open(filename, 'r') as f:
            objects = list(ijson.items(f, 'series'))[0][0]['data']
            v = [int(i[1]) for i in objects]
            v.insert(0,s)
            values.append(v)
    
    if series[k][0] == 'total_production':
        prod = pd.DataFrame(values, columns=years)
    if series[k][0] == 'total_renewable_production':
        ren = pd.DataFrame(values, columns=years)
    if series[k][0] == 'total_consumption':
        con = pd.DataFrame(values, columns=years)
    if series[k][0] == 'total_population':
        pop = pd.DataFrame(values, columns=years)
        
df = pd.concat([prod, ren, con, pop], axis=1, join='inner')

In [146]:
df.shape

(50, 335)

In [121]:
df.head()

,State,2014_total_production,2013_total_production,2012_total_production,2011_total_production,2010_total_production,2009_total_production,2008_total_production,2007_total_production,2006_total_production,...,1969_total_population,1968_total_population,1967_total_population,1966_total_population,1965_total_population,1964_total_population,1963_total_population,1962_total_population,1961_total_population,1960_total_population
0,AL,1353725,1463647,1433370,1400108,1419613,1449279,1482813,1408121,1413506,...,3440,3446,3458,3464,3443,3395,3358,3323,3316,3274
1,AK,1475129,1513859,1563102,1641980,1738207,1853208,1932646,2051639,2100329,...,296,285,278,271,271,263,256,246,238,229
2,AZ,635050,594994,598039,617956,580948,564089,577794,539635,512183,...,1737,1682,1646,1614,1584,1556,1521,1471,1407,1321
3,CA,2413494,2390424,2334863,2634789,2532205,2496181,2522288,2619905,2801850,...,19711,19394,19176,18858,18585,18151,17668,17072,16497,15870
4,CO,3041634,2838193,2921385,2750097,2561459,2497400,2466361,2359170,2289951,...,2166,2120,2053,2007,1985,1970,1936,1899,1844,1769


In [138]:
years = range(1960,2015)
for y in years:
    df[str(y)+'_percent_total_production'] = df[str(y)+'_total_renewable_production'] / df[str(y)+'_total_production']
    df[str(y)+'_renewable_contribution'] = df[str(y)+'_total_renewable_production'] / df[str(y)+'_total_consumption']
df['renewable_change'] = df['2014_renewable_contribution'] / df['1960_renewable_contribution']

In [139]:
df.shape

(50, 335)

In [140]:
df.to_csv('US_Energy_Production.csv', sep=',')
prod.to_csv('total_production.csv')
ren.to_csv('total_renewable_production.csv')
con.to_csv('total_consumption.csv')
pop.to_csv('total_population.csv')